<a href="https://colab.research.google.com/github/andreykoz82/KPI-PBI/blob/master/pycaret_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PYCARET SALES PREDICTION**

In [1]:
%%capture
!pip install --pre pycaret
!pip install xgboost==1.6.2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os
os.chdir("/content/drive/MyDrive/Colab Notebooks/KLS/KPI PBI")

## DATA PREPARATION

In [4]:
import pandas as pd
from dataclasses import dataclass
import numpy as np
import time
from pycaret.time_series import TSForecastingExperiment


def load_data(filename="sales_extend.xlsx"):
    sales = pd.read_excel(filename)
    return sales

@dataclass
class SalesTimeSeries:
    data: pd.DataFrame
    item: str
    aggregation: str
    start_date: str
    end_date: str
    predictions: float = None
    forecast: float = None
    blender: str = None
    exp: float = None
    final_model: float = None

    def transform(self):
        if self.item != 'All':
          self.data = self.data[self.data['Номенклатура'] == self.item]
        month_mapping = {'Январь': 1, 'Февраль': 2, 'Март': 3, 'Апрель': 4, 'Май': 5, 'Июнь': 6, 'Июль': 7,
                         'Август': 8, 'Сентябрь': 9, 'Октябрь': 10, 'Ноябрь': 11, 'Декабрь': 12, }
        month = self.data['По месяцам'].str.replace(' г.', "").str.split(' ', expand=True)[0].map(month_mapping)
        year = self.data['По месяцам'].str.replace(' г.', "").str.split(' ', expand=True)[1]
        df = month.to_frame().join(year)
        df[0] = df[0].astype('str')
        df[1] = df[1].astype('str')
        df['date'] = df[0] + '-' + df[1]
        df['date'] = pd.to_datetime(df['date'])
        self.data['По месяцам'] = df['date']
        self.data = self.data.set_index('По месяцам')['Количество']
        self.data = self.data[(self.data.index >= self.start_date) & (self.data.index <= self.end_date)]
        self.data = self.data.groupby(pd.Grouper(freq=self.aggregation)).sum()
    
    def train_models(self):
      fh = 12
      fold = 5
      self.exp = TSForecastingExperiment()
      self.exp.setup(data=self.data, fh=fh, fold=fold, session_id=42)
      # Get the 3 best baseline models 
      best_baseline_models = self.exp.compare_models(n_select=3)
      compare_metrics = self.exp.pull()
      # Tune best models
      best_tuned_models = [self.exp.tune_model(model) for model in best_baseline_models]
      # Blend best models
      # Get model weights to use
      top_model_metrics = compare_metrics.iloc[0:3]['MAPE']
      top_model_weights = 1 - top_model_metrics/top_model_metrics.sum()
      self.blender = self.exp.blend_models(best_tuned_models, method='voting', weights=top_model_weights.values.tolist())

    def predict(self):
      self.predictions = self.exp.predict_model(self.blender) 

    def make_forecast(self):
      self.final_model = self.exp.finalize_model(self.blender)
      self.forecast = self.exp.predict_model(self.final_model)

    def plot_predictions(self):
      self.exp.plot_model(estimator=self.blender)

    def plot_forecast(self):
      self.exp.plot_model(self.final_model)

In [5]:
sales = load_data()

In [6]:
actual_items = pd.read_excel('actual_items.xlsx')
actual_items.head()

,item,production_line
0,All,2 stage
1,Аир корневища 75г,Линия ББЛ
2,Алтей корни 75г,Линия ББЛ
3,Багульник болотный побеги 50г,Линия ББЛ
4,Береза почки 50г,Линия ББЛ


In [7]:
from ipywidgets import widgets

selected_item = widgets.Dropdown(
    options=actual_items.item.unique(),
    value='Аир корневища 75г',
    description='Item:',
)
selected_item

Dropdown(description='Item:', index=1, options=('All', 'Аир корневища 75г', 'Алтей корни 75г', 'Багульник боло…

In [15]:
temp = SalesTimeSeries(data=sales, item=selected_item.value, aggregation='M', start_date='2012-01-01', end_date='2022-10-01')
temp.transform()
temp.train_models()

,Description,Value
0,session_id,42
1,Target,Количество
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(130, 1)"
5,Transformed data shape,"(130, 1)"
6,Transformed train set shape,"(118, 1)"
7,Transformed test set shape,"(12, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
theta,Theta Forecaster,1.0131,1.0076,598897.7784,801932.2119,0.1846,0.1564,0.6022,0.0680
huber_cds_dt,Huber w/ Cond. Deseasonalize & Detrending,1.1245,1.1199,663072.3509,890595.1882,0.2061,0.1715,0.5165,0.1860
exp_smooth,Exponential Smoothing,1.1257,1.1294,667350.7828,899059.9284,0.2045,0.1706,0.4853,0.1120
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.1408,1.1020,674199.3562,875315.5994,0.2087,0.1723,0.5096,0.1720
lightgbm_cds_dt,Light Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.1424,1.0987,675491.3544,872722.9653,0.2040,0.1736,0.5232,0.1120
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.1438,1.1047,676429.7709,878036.9193,0.2101,0.1734,0.5087,0.1080
rf_cds_dt,Random Forest w/ Cond. Deseasonalize & Detrending,1.1674,1.1015,689518.9565,874042.9325,0.2105,0.1756,0.5241,0.9900
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.1675,1.1067,687957.1149,875717.5757,0.2109,0.1754,0.5149,0.9340
en_cds_dt,Elastic Net w/ Cond. Deseasonalize & Detrending,1.1682,1.1099,686484.2885,878498.8869,0.2093,0.1758,0.5242,0.1020
llar_cds_dt,Lasso Least Angular Regressor w/ Cond. Deseasonalize & Detrending,1.1682,1.1098,686467.9117,878481.2005,0.2093,0.1758,0.5243,0.1400


Processing:   0%|          | 0/123 [00:00<?, ?it/s]

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2016-10,1.7733,1.5578,938444.6361,1116933.5559,0.2599,0.2185,0.4969
1,2017-10,1.0659,1.1648,648592.5160,925469.0734,0.1687,0.1499,0.3746
2,2018-10,0.4702,0.4389,295829.6732,367062.1709,0.0747,0.0739,0.8908
3,2019-10,0.7552,0.7810,481279.8660,676289.1678,0.1678,0.1435,0.7702
4,2020-10,1.0008,1.0959,630342.2004,923907.0914,0.2520,0.1962,0.4785
Mean,NaT,1.0131,1.0076,598897.7784,801932.2119,0.1846,0.1564,0.6022
SD,NaT,0.4341,0.3768,211865.3520,258558.7752,0.0675,0.0499,0.1948


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    4.9s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    5.4s finished


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2016-10,1.6704,1.5071,883994.7125,1080616.0670,0.2553,0.2109,0.5291
1,2017-10,0.9745,1.1999,592981.0874,953351.2171,0.1634,0.1351,0.3363
2,2018-10,0.5051,0.4548,317765.8516,380394.2453,0.0766,0.0752,0.8827
3,2019-10,0.7676,0.7755,489213.2426,671569.6892,0.1747,0.1468,0.7734
4,2020-10,1.0203,1.1535,642616.5263,972484.3702,0.2589,0.1990,0.4222
Mean,NaT,0.9876,1.0182,585314.2841,811683.1178,0.1858,0.1534,0.5887
SD,NaT,0.3869,0.3651,186209.5052,254516.6748,0.0674,0.0488,0.2076


Processing:   0%|          | 0/6 [00:00<?, ?it/s]

In [22]:
temp.predict()
temp.make_forecast()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,EnsembleForecaster,1.2734,1.2385,809458.5046,1076109.4710,0.4072,0.2794,0.0945


In [23]:
temp.plot_predictions()

In [24]:
temp.plot_forecast()